## Agent NB dev -1

- https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/multiagent_notebook.ipynb

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 6.1 MB/s eta 0:00:00


In [16]:
！pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)

In [4]:
# test

print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [14]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool, DuckDuckGoSearchTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [7]:
model

In [9]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [13]:
# from google.colab import userdata
# import os
# os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [17]:
agent = CodeAgent(
    model=model,
    #tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [18]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search("Batman filming locations")                                                        
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
Matt Reeves' The Batman is set in Gotham City, and here are the real-life filming locations around the world used 
to bring the fictional city to life. The Batman was a worldwide production. ... Where The Batman Filmed (All Gotham
Real-Life Locations) By Cooper Hood. Published Mar 4, 2022.

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[Where Was The Batman Filmed? All the Iconic Locations 
Explored](https://otakukart.com/where-was-the-batman-filmed-all-the-iconic-locations-explored/)
The Batman, directed by Matt Reeves, is a gritty, noir-infused take on Gotham City, featuring Robert Pattinson as 
the latest incarnation of the Dark Knight.While the film's setting feels uniquely atmospheric, the shooting 
locations are scattered across the United Kingdom and Chicago, blending real-world settings with digital 
enhancements to bring Gotham to life.

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
Filming on real locations helped create this new version of Gotham. In the past, movies with Batman have shown 
Gotham in different ways. Gotham is usually imagined as a part of New Jersey in the comics. ... It was established 
in 1863. This location was also used to film Batman and Catwoman's motorcycle chase. Anfield Cemetery, Liverpool 
...

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The filming of The Batman film started in January 2020 under the working title Vengeance and wrapped on March 21, 
2021. The second unit filming took place from 17 September 2020 to 13 March 2021. In the below article, we will see
the different locations where The Batman film was filmed. Here is a guide to all the locations of The Batman 
Filming ...

[These Are the Cities Standing in for Gotham in 'The Batman' - 
Distractify](https://www.distractify.com/p/the-batman-filming-locations)
The latest film centered around the Caped Crusader, The Batman, is finally in theaters.One of the key elements of 
any good Batman story is the Gotham City setting. Over the years, various directors have chosen different cities to
stand in for the crime-riddled location, but where did Matt Reeves set his epic Batman tale?

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-fil

...

[Step 4: Duration 14.71 seconds| Input tokens: 18,381 | Output tokens: 1,290]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Manually specify the coordinates                                                                               
  coordinates = {                                                                                                  
      "London": (51.50735, -0.12776),                                                                              
      "Liverpool": (53.40837, -2.99157),                                                                           
      "Glasgow": (55.86424, -4.25181),                                                                             
      "Chicago": (41.87811, -87.62980)                                                                             
  }                                                                                                                
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Calculate travel times for Batman filming locations                                                            
  batman_travel_times = {}                                                                                         
  for location, coords in coordinates.items():                                                                     
      travel_time = calculate_cargo_travel_time(coords, gotham_coords)                                             
      batman_travel_times[location] = travel_time                                                                  
                                                                                                                   
  # Search for supercar factories                                                                                  
  supercar_factories = web_search("supercar factories")                                                            
  print(supercar_factories)                                                                                        
                                                                                                                   
  # Extract coordinates for supercar factories                                                                     
  import re                                                                                                        
  supercar_coordinates = {}                                                                                        
  for line in supercar_factories.split('\n'):                                                                      
      match = re.search(r"(\d+\.\d+),\s*(-?\d+\.\d+)", line)                                                       
      if match:                                                                                                    
          lat, lon = float(match.group(1)), float(match.group(2))                                                  
          supercar_coordinates[line] = (lat, lon)                                                                  
                                                                                                                   
  # Calculate travel times for supercar factories                                                                  
  supercar_travel_times = {}                                                                                       
  for factory, coords in supercar_coordinates.items():                                                             
      travel_time = calculate_cargo_travel_time(coords, g

Execution logs:
## Search Results

[Top 10 Supercar Manufacturers In The World Today - Ape To 
Gentleman](https://www.apetogentleman.com/top-10-supercar-brands/)
Porsche might not be a traditional supercar manufacturer, but over the years the German marque has continually 
flexed its engineering muscles by creating some of the very best supercars ever conceived. The first could arguably
be called the original everyday supercar - the 959 (1986).

[List of exclusively sports car manufacturers - 
Wikipedia](https://en.wikipedia.org/wiki/List_of_exclusively_sports_car_manufacturers)
This is a list of specialist manufacturers or marques of modern and classic sports cars. It includes only companies
that are devoted exclusively to producing sports cars. A sports car is an automobile designed for performance 
driving; however the exact definition is subject to debate.

[All Automotive Brands | Full List of Carmakers - Supercars.net](https://www.supercars.net/blog/all-brands/)
Pagani Automobili S.p.A. is an Italian manufacturer of Supercars and carbon fibre. The company was founded in 1992 
by the Argentinian Horacio Pagani, and is based in San Cesario sul Panaro, near Modena, Italy. gani's first model, 
the Zonda, is propelled by mid-mounted V-12 DOHC engines manufactured by Mercedes-Benz's AMG division.

[Vroom! Touring Italy's Supercar Factories - The New York 
Times](https://www.nytimes.com/2025/03/14/travel/motor-valley-italy-supercars.html)
Supercars, of course, raise ethical dilemmas simply by existing, and touring factories that produce the 
gas-guzzling machines in the age of climate change can feel out of touch.

[10 Car Brands That Build The Most Powerful 
Supercars](https://www.hotcars.com/car-brands-that-build-most-powerful-supercars/)
Czinger Vehicles Inc., commonly known as Czinger is a Los Angeles-based supercar manufacturer founded and 
established in 2019. They utilized AI and 3D printing to develop their first and currently only vehicle, the 
Czinger 21C, which is a hybrid supercar.Production of this cutting-edge American sports car began in 2021 and 
deliveries are expected to start this year.

[Inside the McLaren supercar factory: a behind-the-scenes look at 
...](https://supercarblondie.com/inside-the-mclaren-supercar-factory-in-woking-england/)
Starting with the McLaren F1, its supercar range took off with the MP4-12C, and it has since spawned all-time 
greats like the P1, Senna, and 570S. The supercar side operates alongside the long-time Formula 1 team that secured
its first world constructor's title since 1998 in 2024. Incredibly, the factory produces about 22 supercars each 
day.

[Supercars manufacturing, Italian supercars manufacturing suppliers ...](https://www.supercarsmanufacturing.com/)
2021 - 2024 Sly Garage Factory S.r.l. Supercars Manufacturing Lucca, Italy SGF Manufacturing Facilities Via del 
Marginone 64, 55012 Capannori, Lucca - Italy Tel. in Italy +39.333.6371.644 - Tel. in China +86.155.0227.6780

[Top 10 Super Car Manufacturers - 
Profestyle](https://profestyle.com/top-10-super-car-manufacturers-in-the-world-elite-picks/)
What Makes These Supercar Manufacturers Special? These car makers are more than factories. They are homes where 
dreams turn into real, fast, beautiful cars. They give people speed, style, and a feeling of flying. Each of these 
brands has a story, a soul, and a heart that beats fast.

[Supercars, hypercars, super saloons, GTs, hot hatches, sports cars 
...](https://www.supercarworld.com/cgi-bin/index.cgi)
Click on any manufacturer logo throughout the site to show a brief history, model list and performance stats and 
model timeline for that manufacturer. Compare each manufacturers latest supercars, super saloons, GTs and hot 
hatches for 2021. Show all manufacturers

[Top Luxury Supercar Dealerships, Tuning Studios Brands List - 
Royist](https://www.royist.com/signature-guides/luxury-brands-collection/supercars/)
Explore the top luxury international supercar brands list. Exo

[Step 5: Duration 22.79 seconds| Input tokens: 31,594 | Output tokens: 2,887]